In [52]:
import pandas as pd
import glob
from inc.indicators import apply_all_indicators

def load_crypto_data(symbol="BTC-USD", folder_path="data/crypto"):
    """
    Loads and concatenates CSV files for a given crypto symbol.
    
    Parameters:
        symbol (str): The symbol to load (default "BTC-USD").
        folder_path (str): Path to the folder containing CSV files.
    
    Returns:
        pd.DataFrame: Combined DataFrame sorted by datetime.
    """
    pattern = f"{folder_path}/{symbol.replace('-', '_')}_*.csv"
    files = glob.glob(pattern)

    if not files:
        raise ValueError(f"No files found for {symbol} in {folder_path}")

    dfs = [pd.read_csv(file) for file in files]
    df = pd.concat(dfs, ignore_index=True)

    df['Datetime'] = pd.to_datetime(df['Datetime'])
    df = df.sort_values(['Datetime']).reset_index(drop=True)  # <- Keep all rows, just sort

    return df

def generate_signal(row, rsi_buy_threshold=40, rsi_sell_threshold=65):
    if row['Crossover'] == 'Bullish' and row['RSI'] < rsi_buy_threshold:
        return 'BUY'
    elif row['Crossover'] == 'Bearish' and row['RSI'] > rsi_sell_threshold:
        return 'SELL'
    else:
        return 'HOLD'


def backtest_signals(df, initial_balance=1000):
    """
    Simulates trading based on generated signals.

    Parameters:
        df (pd.DataFrame): DataFrame with 'Signal' and 'Close' columns.
        initial_balance (float): Starting cash balance.

    Returns:
        float: Final account balance after simulation.
        list: History of account values for plotting if needed.
    """
    balance = initial_balance
    position = 0  # Number of coins held
    account_history = []

    for i, row in df.iterrows():
        price = row['Close']
        signal = row['Signal']

        if signal == 'BUY' and balance > 0:
            position = balance / price  # Buy as much as possible
            balance = 0
        elif signal == 'SELL' and position > 0:
            balance = position * price  # Sell everything
            position = 0

        # Track account value at each step
        account_value = balance + (position * price)
        account_history.append(account_value)

    # Final value (if still holding position, it will be liquidated at last price)
    final_value = balance + (position * df['Close'].iloc[-1])

    return final_value, account_history

def auto_backtest(symbols, folder_path="data/crypto", initial_balance=1000):
    """
    Automatically backtests multiple crypto symbols.

    Parameters:
        symbols (list): List of symbol strings.
        folder_path (str): Path to the folder with CSV files.
        initial_balance (float): Starting cash for each symbol.

    Returns:
        pd.DataFrame: Summary of final balances for each symbol.
    """
    results = []

    for symbol in symbols:
        try:
            df = load_crypto_data(symbol=symbol, folder_path=folder_path)
            df = apply_all_indicators(df)
            df['Signal'] = df.apply(generate_signal, axis=1)
            final_balance, history = backtest_signals(df, initial_balance=initial_balance)

            results.append({
                'Symbol': symbol,
                'Final_Balance': final_balance,
                'Profit': final_balance - initial_balance,
                'Profit_%': ((final_balance - initial_balance) / initial_balance) * 100
            })
        
        except Exception as e:
            print(f"Error processing {symbol}: {e}")

    summary_df = pd.DataFrame(results)
    return summary_df

def live_audit(symbols, folder_path="data/crypto"):
    """
    Provides the latest signal for each symbol for real-time auditing.

    Parameters:
        symbols (list): List of symbols to audit.
        folder_path (str): Where CSV files are stored.

    Returns:
        pd.DataFrame: Latest signal snapshot for each symbol.
    """
    audit_results = []

    for symbol in symbols:
        try:
            df = load_crypto_data(symbol=symbol, folder_path=folder_path)
            df = apply_all_indicators(df)
            df['Signal'] = df.apply(generate_signal, axis=1)

            latest = df.iloc[-1]  # Most recent row
            audit_results.append({
                'Symbol': symbol,
                'Datetime': latest['Datetime'],
                'Close': latest['Close'],
                'Signal': latest['Signal'],
                'RSI': latest['RSI'],
                'MACD': latest['MACD'],
                'Signal_Line': latest['Signal_Line'],
                'MACD_Diff': latest['MACD_Diff']
            })

        except Exception as e:
            print(f"Error auditing {symbol}: {e}")

    audit_df = pd.DataFrame(audit_results)
    return audit_df

# ===
def optimize_rsi_thresholds(df, buy_range=(30, 50, 5), sell_range=(60, 80, 5), initial_balance=1000):
    """
    Finds the best RSI buy/sell thresholds by backtesting different combinations.

    Parameters:
        df (pd.DataFrame): DataFrame with indicators already applied.
        buy_range (tuple): (start, end, step) for buy RSI threshold testing.
        sell_range (tuple): (start, end, step) for sell RSI threshold testing.
        initial_balance (float): Starting balance for backtest.

    Returns:
        dict: Best parameters and corresponding performance.
    """
    best_result = {
        'buy_threshold': None,
        'sell_threshold': None,
        'final_balance': -float('inf')
    }

    for buy_threshold in range(*buy_range):
        for sell_threshold in range(*sell_range):
            temp_df = df.copy()
            temp_df['Signal'] = temp_df.apply(
                lambda row: generate_signal(row, rsi_buy_threshold=buy_threshold, rsi_sell_threshold=sell_threshold),
                axis=1
            )
            final_balance, _ = backtest_signals(temp_df, initial_balance=initial_balance)

            if final_balance > best_result['final_balance']:
                best_result = {
                    'buy_threshold': buy_threshold,
                    'sell_threshold': sell_threshold,
                    'final_balance': final_balance
                }

    return best_result


In [34]:
df= load_crypto_data(symbol="BTC-USD", folder_path="data/crypto")

# After loading your data
df = apply_all_indicators(df)
df['Signal'] = df.apply(generate_signal, axis=1)

df.tail(3)

,Symbol,Datetime,Open,High,Low,Close,Volume,Best_Bid_Size,Best_Ask_Size,Total_Bid_Depth,...,MACD_ROC,VWAP_1m,VWAP_15m,VWAP_1h,VWAP_Day,MACD_Diff,Crossover,Pre_x_Warning,Pre_x_angle,Signal
1286,BTC-USD,2025-04-27 16:05:00-04:00,94335.66,94381.36,94335.66,94380.52,0.552025,0.008659,0.002086,761140.841315,...,-142.763020,94365.846667,94436.144076,94397.977497,NaN,-12.938989,None,None,None,HOLD
1287,BTC-USD,2025-04-27 16:05:00-04:00,94335.66,94394.67,94335.66,94394.66,1.567444,0.073735,0.026808,761140.108098,...,-157.221815,94374.996667,94435.352011,94398.585507,NaN,-10.869629,None,None,None,HOLD
1288,BTC-USD,2025-04-27 16:05:00-04:00,94335.66,94401.80,94335.08,94355.37,2.991859,0.000018,0.005190,761123.960442,...,-205.986853,94364.083333,94433.585449,94398.298979,94245.678777,-11.544344,None,None,None,HOLD


In [35]:
final_balance, history = backtest_signals(df)
print(f"Final Balance: ${final_balance:.2f}")


Final Balance: $999.53


In [57]:
symbols = [
    "BTC-USD", "ETH-USD", "SOL-USD", "LTC-USD",
    "ADA-USD", "AVAX-USD", "DOGE-USD", "MATIC-USD",
    "XRP-USD", "PEPE-USD"
]

summary = auto_backtest(symbols)
print(summary)


      Symbol  Final_Balance     Profit  Profit_%
0    BTC-USD    1002.772942   2.772942  0.277294
1    ETH-USD    1003.100413   3.100413  0.310041
2    SOL-USD    1021.563527  21.563527  2.156353
3    LTC-USD    1015.778647  15.778647  1.577865
4    ADA-USD    1018.167012  18.167012  1.816701
5   AVAX-USD    1000.867175   0.867175  0.086717
6   DOGE-USD    1026.786218  26.786218  2.678622
7  MATIC-USD     985.300122 -14.699878 -1.469988
8    XRP-USD    1047.838298  47.838298  4.783830
9   PEPE-USD     985.557986 -14.442014 -1.444201


In [59]:
audit = live_audit(symbols)
print(audit)


      Symbol                  Datetime         Close Signal        RSI  \
0    BTC-USD 2025-04-27 17:45:00-04:00  94455.480000   HOLD  42.346306   
1    ETH-USD 2025-04-27 17:45:00-04:00   1805.560000   HOLD  43.735225   
2    SOL-USD 2025-04-27 17:45:00-04:00    149.610000   HOLD  67.441860   
3    LTC-USD 2025-04-27 17:45:00-04:00     86.560000   HOLD  45.945946   
4    ADA-USD 2025-04-27 17:45:00-04:00      0.715300   HOLD  43.902439   
5   AVAX-USD 2025-04-27 17:45:00-04:00     22.130000   HOLD  71.428571   
6   DOGE-USD 2025-04-27 17:45:00-04:00      0.181790   HOLD  44.502618   
7  MATIC-USD 2025-04-27 17:45:00-04:00      0.242000   HOLD  50.000000   
8    XRP-USD 2025-04-27 17:45:00-04:00      2.273100   HOLD  54.901961   
9   PEPE-USD 2025-04-27 17:45:00-04:00      0.000009   HOLD  50.000000   

           MACD   Signal_Line     MACD_Diff  
0  3.324071e+01  4.070282e+01 -7.462115e+00  
1  8.728226e-01  1.038911e+00 -1.660885e-01  
2  1.420139e-01  1.290515e-01  1.296238e-02  
3

In [55]:
df = load_crypto_data(symbol="BTC-USD")      # or whatever symbol
df = apply_all_indicators(df)

best_rsi_settings = optimize_rsi_thresholds(df)
print(best_rsi_settings)


{'buy_threshold': 35, 'sell_threshold': 65, 'final_balance': 1003.8666646247513}


In [60]:
for symbol in symbols:
    df = load_crypto_data(symbol)
    df = apply_all_indicators(df)
    best_settings = optimize_rsi_thresholds(df)
    print(f"{symbol}: {best_settings}")


BTC-USD: {'buy_threshold': 40, 'sell_threshold': 70, 'final_balance': 1003.8224872948633}
ETH-USD: {'buy_threshold': 40, 'sell_threshold': 60, 'final_balance': 1003.3778821954576}
SOL-USD: {'buy_threshold': 35, 'sell_threshold': 65, 'final_balance': 1032.6028288667587}
LTC-USD: {'buy_threshold': 40, 'sell_threshold': 65, 'final_balance': 1024.8966248632232}
ADA-USD: {'buy_threshold': 40, 'sell_threshold': 75, 'final_balance': 1030.6783811032694}
AVAX-USD: {'buy_threshold': 40, 'sell_threshold': 60, 'final_balance': 1017.3281069473693}
DOGE-USD: {'buy_threshold': 40, 'sell_threshold': 65, 'final_balance': 1017.0830066091631}
MATIC-USD: {'buy_threshold': 30, 'sell_threshold': 70, 'final_balance': 992.209922099221}
XRP-USD: {'buy_threshold': 40, 'sell_threshold': 65, 'final_balance': 1047.3318292739077}
PEPE-USD: {'buy_threshold': 35, 'sell_threshold': 65, 'final_balance': 1004.5095828635851}
